# Assignment: 1 - Camera Calibration 

### Importing necessary libraries

In [73]:
import numpy as np
import pandas as pd
from scipy import linalg


In [74]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

def pretty_print(x):
    x_shape = x.shape
    for i in range(x_shape[0]):
        for j in range(x_shape[1]):
            if (j==(x_shape[1]-1)):
                print("{:.4f}".format(x[i][j]), end = "")
            else:
                print("{:.4f} & ".format(x[i][j]), end = "")
        print('\\\\')

### Normalization

In [75]:
def Normalization(nd, x):
    # CONVERTING TO NUMPY ARRAY
    x = np.asarray(x)
    # print(x)

    # CALCULATING centroid and mean distance from centroid
    m = np.mean(x, 0)
    dist = np.mean(np.sqrt(np.sum(np.square(x - m))))

    # NORMALIZATION MATRIX FOR WORLD POINTS(3D) AND IMAGE-PIXEL POINTS(2D)
    if nd == 2:
        s2D = np.sqrt(2) / dist
        Tr = np.diag([s2D, s2D, 1])
        Tr[0:2, 2] = -m * s2D

    else:
        s3D = np.sqrt(3) / dist
        Tr = np.diag([s3D, s3D, s3D, 1])
        Tr[0:3, 3] = -m * s3D

    x = np.dot(Tr, np.concatenate((x.T, np.ones((1, x.shape[0])))))
    x = x[0:nd, :].T

    print("MEAN DISTANCE FROM CENTER : {}".format(np.mean(np.sqrt(np.sum(np.square(x))))))

    return Tr, x

### DLT Camera Calibration

In [76]:
def DLT_cam_calib(nd, xyz, img_pt_):

    # Converting all variables to numpy array
    xyz = np.asarray(xyz)
    img_pt_ = np.asarray(img_pt_)

    n = xyz.shape[0]

    print("\n\nNUMBER OF POINTS: {}\n\n".format(n))

    Txyz, xyzn = Normalization(nd, xyz)
    Timg_pt_, img_pt_n = Normalization(2, img_pt_)

    A = []

    for i in range(n):
        x, y, z = xyzn[i, 0], xyzn[i, 1], xyzn[i, 2]
        u, v = img_pt_n[i, 0], img_pt_n[i, 1]
        A.append([x, y, z, 1, 0, 0, 0, 0, -u * x, -u * y, -u * z, -u])
        A.append([0, 0, 0, 0, x, y, z, 1, -v * x, -v * y, -v * z, -v])

    # Convert A to array
    A = np.asarray(A)

    # Find the 11 parameters:
    U, S, V = np.linalg.svd(A)

    # The parameters are in the last line of Vh and normalize them
    L = V[-1, :] / V[-1, -1]

    # Camera projection matrix
    H = L.reshape(3, nd + 1)

    # Denormalization
    H_temp = np.dot(np.linalg.pinv(Timg_pt_), H)
    H = np.dot(H_temp, Txyz)
    H = H / H[-1, -1]
    L = H.flatten()

    # Mean error of the DLT (mean residual of the DLT transformation in units of camera coordinates):
    img_pt_homo = np.concatenate((xyz.T, np.ones((1, xyz.shape[0]))))
    img_pt_2 = np.dot(H, img_pt_homo)
    img_pt_2 = img_pt_2 / img_pt_2[2, :]

    # Mean distance:
    projected_pts = img_pt_2.T
    actual_pixels = img_pt_

    # Printing teh actual and the estimated points
    print('\n\nACTUAL PTS   |             ESTIMATED PTS')
    for i in range(len(img_pt_2.T)):
        print("{}   <---->  {}".format(actual_pixels[i], projected_pts[i][0:2]))

    err = np.sqrt(np.mean(np.sum(np.square((img_pt_2[0:2, :].T - img_pt_)), 1)))

    return L, err, projected_pts[:,0:2]

In [77]:
def get_intrest_points():

    # WORLD CORDINATES
    data = pd.read_excel("dataset.xlsx")
    x = np.array(data['X'])
    y = np.array(data['Y'])
    z = np.array(data['Z'])

    # IMAGE PIXEL COORDINATES
    u = np.array(data['x'])
    v= np.array(data['y'])

    # LIST OF WORLD PTS AND CORRESPONDING PIXEL PTS
    world_pts = []
    pixel_pts = []

    for i in range(9):
        world_pts.append([x[i], y[i], z[i]])
        pixel_pts.append([u[i], v[i]])

    print("DATA FRAME")
    print(data)

    # from sklearn.utils import shuffle
    # world_pts, pixel_pts = shuffle(world_pts, pixel_pts, random_state = 0)

    print("\n\nWORLD points")
    print(world_pts)
    return world_pts, pixel_pts
   

### RQ Decomposition 

In [78]:
def RQ_decomposition(P):
    M = P[0:3,0:3]
    print("\nM:\n",M)
    K, R = linalg.rq(M)
    T = np.diag(np.sign(np.diag(K)))
    print(T)

    K = np.dot(K, T)
    R = np.dot(T, R)
    C = np.dot(linalg.inv(-M), P[:, 3])
    return K, R, C


In [79]:
    def camera_param(P):
        print("\nP:\n",P)

        K, R, C = RQ_decomposition(P)

        print("\n\n K MATRIX: ")
        print(K)
        #pretty_print(K)

        print("\n\n R MATRIX: ")
        print(R)
        #pretty_print(R)

        print("\n\n Camera center: ")
        print(C)
        #pretty_print(C)

        print("\n\n Normalized camera matrix: ")
        print(K/K[2][2])
        #pretty_print(K/K[2][2])


In [80]:
#camera_param(P)

In [81]:

xyz, img_pt_ = get_intrest_points()
nd = 3
P, err, projected_pts = DLT_cam_calib(nd, xyz, img_pt_)
P = P.reshape(3,4)

DATA FRAME
       X     Y   Z    x     y  Unnamed: 5
0   20.0   0.0   0  708   991           1
1    0.0  20.0   0  213   998           2
2    0.0   0.0  20  455   597           3
3   20.0  20.0   0  464  1192           4
4   20.0   0.0  20  727   658           5
5    0.0  20.0  20  182   665           6
6    0.0  15.0  10  279   810           7
7    0.0   7.5  15  373   699           8
8   12.5  17.5   0  396  1082           9
9    5.0   0.0   5  516   828          10
10  10.0  10.0   0  463   998          11
11  10.0   0.0  10  576   779          12
12   0.0  10.0  10  343   784          13
13   0.0   0.0   0  463   874          14
14  15.0  10.0   0  529  1038          15
15  15.0   0.0  10  642   805          16
16  25.0   0.0  25  829   576          17
17   0.0  25.0  25   76   586          18


WORLD points
[[20.0, 0.0, 0], [0.0, 20.0, 0], [0.0, 0.0, 20], [20.0, 20.0, 0], [20.0, 0.0, 20], [0.0, 20.0, 20], [0.0, 15.0, 10], [0.0, 7.5, 15], [12.5, 17.5, 0]]


NUMBER OF POINTS: 9


ME

### Projection matrix (P)

In [82]:
print('Matrix')
print(P)
#pretty_print(P)

Matrix
[[5.718 -14.406 -1.994 461.588]
 [-3.344 -3.377 -15.986 872.241]
 [-0.009 -0.010 -0.004 1.000]]


### RMSE

In [83]:
print('\nError')
print(err)


Error
1.734339964635972


### Camera Parameters

In [84]:
camera_param(P.reshape(3,4))


P:
 [[5.718 -14.406 -1.994 461.588]
 [-3.344 -3.377 -15.986 872.241]
 [-0.009 -0.010 -0.004 1.000]]

M:
 [[5.718 -14.406 -1.994]
 [-3.344 -3.377 -15.986]
 [-0.009 -0.010 -0.004]]
[[1.000 0.000 0.000]
 [0.000 -1.000 0.000]
 [0.000 0.000 1.000]]


 K MATRIX: 
[[14.108 0.114 6.720]
 [0.000 14.106 8.897]
 [0.000 0.000 0.014]]


 R MATRIX: 
[[0.722 -0.692 -0.005]
 [0.184 0.198 -0.963]
 [-0.667 -0.694 -0.270]]


 Camera center: 
[45.613 45.237 35.464]


 Normalized camera matrix: 
[[1007.552 8.169 479.973]
 [0.000 1007.443 635.453]
 [0.000 0.000 1.000]]
